In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>", fig.align = "center",
  eval=TRUE
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

# Install additional dependencies for this vignette
BiocManager::install("gghighlight")
BiocManager::install("batchelor")
BiocManager::install("spatialDE")

packageVersion("Voyager")

# Introduction

In [ ]:
library(Voyager)
library(SingleCellExperiment)
library(SpatialExperiment)
library(SpatialFeatureExperiment)
library(batchelor)
library(scater)
library(scran)
library(bluster)
library(glue)
library(purrr)
library(tidyr)
library(dplyr)
library(ggplot2)
library(gghighlight)
library(patchwork)
library(spdep)
library(spatialDE)
library(BiocParallel)

theme_set(theme_bw())

# Dataset

The dataset used in this vignette is from the paper [Strategies for Accurate Cell Type Identification in CODEX Multiplexed Imaging Data](doi.org/10.3389/fimmu.2021.727626)(Hickey, et.al 2021). The data were collected as part of the HuBMap consortium which seeks to characterize healthy human tissues and make data  broadly available. More specifically, this dataset characterizes 4 regions of the large intestine (colon) from a single donor. This vignette will focus on data from the sigmoid colon. 

The intestinal sections were interrogated using the multiplexed imaging method CO-Detection by indEXing (CODEX). CODEX involves cyclical staining of a tissue with DNA-barcoded antibodies. At each round of experimentation, fluoresently labeled probes hybridize to the tissue bound DNA-conjugated antibodies are subsequently imaged and the stripped from the tissue. At present, the technology quantifies up to 60 markers in a single experiment. Raw images generated from this process are subjected to image stitching, drift compensation, deconvolution, and cycle concatenation using publicly avaialable [software](https://github.com/nolanlab/CODEX). The result of this pre-processing is a matrix that contains the location of individual cells and the quantified markers for each cell. Cell types were assigned as described in the manuscript linked above. Briefly, the authors used a hand-gating strategy to define cell types and create a standard to compare the effect of normalization methods on clustering and cell annotation. 

The raw intensity data are available for download from HuBMAP with identifier [HBM575.THQMM.284](https://portal.hubmapconsortium.org/browse/dataset/e6fd525b837f4cf736c8af830f4f750f) and the cell type annotations are provided as supplementary data in the manuscript. The data relevant to this vignette have been converted to a `SFE` object and are available to download [here](https://caltech.box.com/public/static/zfr8l20450n2z28lnp0ugdj471ph9eyx) from Box.  

These data will be submitted to the `SFEData` package on Bioconductor and will be available there in a future release. 

We will begin by downloading the data and loading it in to R. 

In [ ]:
download.file("https://caltech.box.com/public/static/zfr8l20450n2z28lnp0ugdj471ph9eyx",'./codex.Rds', mode='wb', method = 'wget', quiet = TRUE)

sfe <- readRDS("./codex.Rds")
sfe

The rows in the count matrix correspond to the 47 barcoded genes measured by CODEX. Additionally, the authors provide some metadata for the cells, including the cell type. 

It turns out the column names are not unique which will cause errors in downstream analysis. We will update the column names below

In [ ]:
cellids <- glue("{colData(sfe)$fn}_{colData(sfe)$cell_id}")
colnames(sfe) <- cellids

## Exploratory Data Analysis

In [ ]:
celldensity <- plotCellBin2D(sfe)

celldensity

We can see from the figure above that the colonic epithelium is enriched with cells while the loose connective tissue and muscle layers beneath the epithelial layer are more sparsely populated. This is in line with known colon histology. The epithelium is enriched with goblet cells and has invaginations that project inwards towards the connective tissue. Smooth muscle cells are also prominent in the colon, where bands of muscle contract to move colonic contents towards the rectum.  

We can visualize these cell types in space using the `plotSpatialFeature()` function. We will highlight Goblet and smooth muscle cells to display their relative distribution in the tissue. Since CODEX image processing relies on segmentation, each dot in the plot represents a single cell. Here, each cell is represented by its centroid, but can also be visualized as cell polygons in cases where the segmentation mask is available.  

In [ ]:
spatial <- plotSpatialFeature(sfe, features='cell_type', colGeometryName = "centroids") +
    gghighlight(cell_type %in% c("Goblet", "SmoothMuscleME"))

spatial

The goblet cells clearly define the epithelial border of the tissue and the thick bands of smooth muscle cells are prominent below the mucosa. 

Next, we will compute some gene level metrics for each of the 47 barcoded genes. In contrast to RNA-based methods, the fields in the matrix represent intensities rather than counts.  

In [ ]:
rowData(sfe)$mean <- rowMeans(assay(sfe))
rowData(sfe)$var <- rowVars(assay(sfe))

data.frame(rowData(sfe)) |>
    ggplot(aes(mean, var)) + 
    geom_point()

There appears to be a sigmoid relationship between the mean and variance of the protein expression. The pattern is reminiscent of what might be expected if the intensity values were derived from a Gamma distribution, the continuous analog of the Negative Binomial distribution that is typically used to describe count data from scRNA-seq experiments. This may have implications for how CODEX data is variance stabilized in the future. 

CODEX data is subject to noise from several sources including segmentation artifacts, nonspecific staining, and imperfect tissue processing. These are factors that can limit accurate quantification of signal intensity and impede accurate cell annotation. The authors of the dataset tested the effects of several normalization methods on cell type annotation and clustering and found that Z-score normalization of each marker resulted in accurate identification of both rare and common cell types. In the cell below, we demonstrate how to accomplish this using standard matrix operations. The normalized count matrix is typically stored in the `logcounts` slot for scRNA-seq data, but we will instead store the normalized matrix in a slot called `normalizedIntensity`. 

In [ ]:
mtx <- assay(sfe, 'protein')
assay(sfe, 'normalizedIntensity') <- (mtx - rowMeans(mtx))/rowSds(mtx)

assays(sfe)

# Spatial EDA

Neighbor definition is a critical step in computation of metrics of spatial dependency like Moran's I and Geary's C. The definition of neighbors is complex, even when cell polygons are available. In the latter case, the `poly2nb` method might be appropriate to assign two cells as neighbors if they physically touch each other or share a border. This may not be tenable in cases where cells are sparse or cells are represented by their centroids, as in this dataset. 

We will compute the spatial neighborhood graph using the `knearestneigh` function as it is implemented in `spdep`. In brief, Euclidean distances are computed between each pair of cells and the `k` nearest cells are considered neighbors. In the following code cell, we will consdier `k=10` for speed purposes, but this may not be ideal in general. 

The weights of the neighborhood matrix are inverse-distance weighted, such that the the weight of regions listed as neighbors increases as the distance between pairs of points decreases. Setting `style = "W"` ensures that the weights are row standardized. 

In [ ]:
colGraph(sfe, "knn10") <- findSpatialNeighbors(
    sfe, method = "knearneigh", dist_type = "idw", 
    k = 10, style = "W")

The `plotColGraph()` function plots the graph in space along with its corresponding `colGeometry`, but since there are so many cells in this dataset, plotting the neighborhood graph may not be as useful as many connections will be obscure by overlapping lines. In any case, we will demonstrate use of the function below. 

In [ ]:
plotColGraph(sfe, colGraphName = "knn10", colGeometryName = 'centroids')

Next, we will explore univariate metrics for global spatial autocorrelation. Since few genes are quantified in this study, we will compute the metrics for all genes. For larger datasets, it may be useful to restrict analysis to the most variable genes. 

We use the `runUnivariate()` function to compute the spatial autocorrelation metrics and save the results in the SFE object. 

In [ ]:
sfe <- runUnivariate(
    sfe, type = "moran.mc", features = rownames(sfe),
    exprs_values = "normalizedIntensity", colGraphName = "knn10", nsim = 100,
    BPPARAM = MulticoreParam(2))

sfe <- runUnivariate(
    sfe, type = "moran.plot", features = rownames(sfe),
    exprs_values = "normalizedIntensity", colGraphName = "knn10")

The results of these computations are accessible in the `rowData` attribute of the SFE object. 

In [ ]:
colnames(rowData(sfe))

Next, we plot the results of the genes with the highest Moran's I statistic. 

In [ ]:
top_moran <- data.frame(rowData(sfe)) |>
    arrange(desc(moran.mc_statistic_sample01)) |>
    head(6) |> 
    rownames()

moran <- plotMoranMC(sfe, features = top_moran, facet_by = 'features')
moran

The vertical line in each plot represents the observed Moran's I while the density represents the Moran's I statistic for each of the random permutations of the data. Each of these plots suggests that the Moran's I statistic is significant. We can  plot the normalized intensity for these genes in space. 

In [ ]:
plotSpatialFeature(
    sfe, features=top_moran, colGeometryName = "centroids",
    exprs_values = "normalizedIntensity", scattermore = TRUE, pointsize = 1)

While most of these genes appear to have some spatial distribution, it also seems that it may overlap with cell type. The cells that appear to express the genes of interest seem to be spatially restricted to known boundaries in the tissue. 

The `moranPlot()` function plots spatial data against its spatially lagged values and enables users to assess how similar observed values are to its neighbors. When the variable is centered, the plot is divided into four quadrants defined by the horizontal line y = 0 and the vertical line x = 0. Points in the upper right (or high-high) and lower left (or low-low) quadrants indicate positive spatial association, and points in the lower right (or high-low) and upper left (or low-high) quadrants include observations that exhibit negative spatial association.

In [ ]:
moranPlot(sfe, top_moran[1])

# Differential Expression

While Moran's I and other global spatial autocorrelation metrics provide insight to the spatial patterns of gene expression, it is necessarily limited by the structure imposed by the spatial weights matrix. A complimentary task might be to identify spatially variable (SV) genes. One such method to do this is described in [SpatialDE: identification of spatially variable genes](). The method described in the manuscript relies on Gaussian process regression and decomposes variability in expression into spatial and non-spatial components. In contrast to Moran's I, the covariance between each pair of cells is modeled as a function of the distance between them. Notably, it does not require an explicit specification of hte neighborhood graph, but rather the a parameter controls the decay in covariance as distance increases. 

The `spatialDE` package is implemented in R and requires a normalized matrix as input. The `spatialDE()` function from the package performs normalization steps before running the algorithm. Because the data has already been normalized, we will use the `run()` function directly to run spatialDE. We will first have to convert the centroid coordinates to a data frame as required by the function.

In [ ]:
# Store coordinates in a data frame object
coords <- centroids(sfe)$geometry |>
    purrr::map_dfr(\(x) c(x = x[1], y = x[2]))

# de_res <- spatialDE::run(assay(sfe,"normalizedIntensity"), coords, verbose=TRUE)

We can plot the normalized expression of the top 5 genes in space.

In [ ]:
# top_genes <- de_res |>
#   arrange(pval) |>
#   slice_head(n=6) |>
#   pull(g)
# 
# plotSpatialFeature(sfe, top_genes, colGeometryName="centroids",
#                    exprs_values = "normalizedIntensity")

Perhaps unsurprisingly, the expression of the top DE genes seems to highlight the spatial distribution of known cell types in the tissue rather than identify spatially restricted gene expression. This is related to the experimental design where the targeted genes were chosen to differentiate cell types. Perhaps in genome-wide technologies, the potential for discovery of neew gene expression patterns is more plausible. There is an open question as to whether these results offer new information compared to what is inferred by typical DE expression methods.  

These analyses represent a minority of the types of inferences that can be made from protein expression data. It would be interested to investigate how the protein expression results compare or inform data from spatail scRNA-sequencing experiments. Already, there is work being done to obtain multimodal spatial measurements on the same sample. Importantly however, considerations should be made on the types of biases each individual technology adds to the measurements. These are active areas of research that are ripe for future exploration. 

# Session Info

In [ ]:
sessionInfo()

# References